In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [1]:
%pip install pandarallel
%pip install dialogflow
%pip install gensim==4.2.0
%pip install annoy
%pip install tqdm
%pip install pickle
%pip install stop_words
%pip install pymorphy2
%pip install pyTelegramBotAPI

  Using cached gensim-4.2.0-cp310-cp310-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (24.0 MB)
  Attempting uninstall: gensim
    Found existing installation: gensim 4.3.2
    Uninstalling gensim-4.3.2:
      Successfully uninstalled gensim-4.3.2
  Using cached annoy-1.17.3.tar.gz (647 kB)
  Preparing metadata (setup.py) ... done
  Created wheel for annoy: filename=annoy-1.17.3-cp310-cp310-linux_x86_64.whl size=552451 sha256=4cae90cbc63361059cee2884878dc9016920a875899de3d36157dff83032ac6d
  Stored in directory: /root/.cache/pip/wheels/64/8a/da/f714bcf46c5efdcfcac0559e63370c21abe961c48e3992465a
Successfully built annoy
ERROR: Could not find a version that satisfies the requirement pickle (from versions: none)
ERROR: No matching distribution found for pickle
  Preparing metadata (setup.py) ... done
  Created wheel for stop_words: filename=stop_words-2018.7.23-py3-none-any.whl size=32895 sha256=16dc2114dace5b7c4973215856f1f54553572c1f409534a2743c3fc469ccb05c
  Stored in directory: /root

In [ ]:
#RESTART KERNEL

In [1]:
import os
import string
from pymorphy2 import MorphAnalyzer
from stop_words import get_stop_words
import annoy
from multiprocessing import cpu_count
from gensim.models.word2vec import Word2Vec
from gensim.models.fasttext import FastText
import pickle
import numpy as np
from tqdm import tqdm_notebook
import pandas as pd
import telebot
from telebot import types
from IPython.core.display import HTML
from pandarallel import pandarallel
pandarallel.initialize(progress_bar=True)

INFO: Pandarallel will run on 1 workers.
INFO: Pandarallel will use Memory file system to transfer data between the main process and workers.


In [2]:
df = pd.read_json('/content/drive/MyDrive/recipes_raw/recipes_raw_nosource_fn.json')

In [3]:
df.head()

,p3pKOD6jIHEcjf20CCXohP8uqkG5dGi,S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO,o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK,5l1yTSYFifF/M2dfbD6DX28WWQpLWNK,kRBQSWtqYWqtkb34FGeenBSbC32gIdO,PmoGGX9RC5zbMJXtZH1SKzWKLFX4Aii,g2RYP1spIUlAYsytDMsdfLNQEOnLUrO,94NWE4F4C5NzgH0zYqSdmPM/G55BjKO,HdIbPB/8cgk17hAzbc2jUN4VBn//QKS,mC7Nb4F2S.DMCjZ39kzS/ak9GqEp4iO,...,CUZuiIs2F5A4u4EzEXB/yHN38So5CRy,08JThnwO3qzkhSKVuef6Ehg.HVPXt7C,0jaoQnb1iG0bnScGJOt/zmAl6lsOh6y,6Qp4fKIuJAkrukUvmccA8qWrezzLwFq,xwEq9YYRt37I/LTqQUWvAi1uSusxhDm,4bfMWxlbKhx/McJq/89k0SBdw.VvAzW,T8lWBA1fcVdjxhMSWuoAbGoy5Lj.A8m,f/coffo2TMs2J2gq5nTOUIqH2TRAkui,q3aDJc4zoEF5QT4e7Mn.ieQwV.DyHwS,7cXA77UpdDtIfBug2v6lEVIuV3Zcvhm
instructions,Toss ingredients lightly and spoon into a butt...,Watch how to make this recipe.\nSprinkle the s...,"In a large saucepan, let the beans soak in eno...",Watch how to make this recipe.\nPreheat the ov...,Special equipment: sushi mat\nCook the brown r...,Heat a large nonstick skillet over medium-high...,Sprinkle the saffron into 1/4 cup hot water; l...,"If using top-sliced buns, spread softened butt...",Cut around the stems of the peppers (reserving...,Preheat the oven to 450 degrees F. Put the cau...,...,Fill a glass or cocktail shaker with some ice....,Watch how to make this recipe.\nHeat a heavy m...,Preheat the oven to 250 degrees F\nIn a small ...,Bring a small saucepan of water to a boil. Sep...,Grease 8 cream horn metal cones. Cut the puff ...,Watch how to make this recipe.\nPreheat a gril...,Preheat the broiler. Cut the tomatoes in 1/2 c...,Heat the oil and butter in a large skillet ove...,Preheat oven to 350 degrees. On the top half o...,Watch how to make this recipe.\nIn a medium sa...
ingredients,"[1/2 cup celery, finely chopped, 1 small green...","[2 pounds skirt steak, cut into 1/2-inch dice,...","[1 1/2 cups dried black beans, picked over and...","[1 1/4 pounds ground chuck, One 15-ounce can t...","[1 cup rice, brown, medium-grain, cooked, 1/2-...","[1 tablespoon extra-virgin olive oil, 2 baby I...",[1/2 teaspoon lightly crumbled saffron threads...,[4 top--sliced hot-dog buns (or fashion your o...,"[6 green or red bell peppers, 1/2 cup olive oi...","[1 medium head cauliflower (about 2 pounds), c...",...,"[Ice, 1 slice jalapeno, seeded 1 by 1/4-inch p...","[1/4 cup butter, 1/3 cup sugar, 1 teaspoon gro...","[2 cups coconut milk, 1 short stalk lemon gras...","[2 large heads garlic, 1 cut-up chicken, about...","[1 sheet frozen puff pastry, thawed, 1 egg, 1 ...","[4 ears fresh corn, 2 heads Belgian endive, 2 ...","[4 large plum tomatoes, Salt and sugar, 1 1/2 ...","[3 tablespoons olive oil, 2 tablespoons unsalt...","[8 ounces butter, 8 ounces bittersweet chocola...",[3 (10.5-ounce) cans restaurant-style condense...
title,Grammie Hamblet's Deviled Crab,Infineon Raceway Baked Beans,Southwestern Black Bean Dip,Sour Cream Noodle Bake,Sushi Renovation,Middle-Eastern Eggplant Rounds,Saffron Jewel Rice,Maine Lobster Rolls,Stuffed Peppers with Pilaf,Cauliflower with Brown Butter and Crispy Crumbs,...,Spicy Agave Wave,Apple Chimichangas,Coconut-Kaffir Leaf Poached Halibut with Saute...,Chicken Braised with 20 Cloves of Garlic,Cream Horns,Summer Corn Salad,Zucchini Stuffed Tomatoes,Pepper Pasta Quick Cook,Chocolate Cake with Armagnac Ice Cream,Crabby Bisque
picture_link,None,Ja5uaD8Q7m7vvtWwk2.48dr1eCre/qi,None,nm/WxalB.VjEZSa0iX9RuZ8xI51Y7bS,None,ibuqgKBoAYj7a086h/tYaHYu2M4N3pS,OR0cC2/SQlgojJz//n8XpUVphfPUuIi,None,None,None,...,1mj7YsV.BecQWcQXnLqOypoElrAkKXS,cSTbbKHkN5PMJvHd9ieoj3HevTJS9re,None,None,None,G9XPV1I0nULIbwS76f..0AIq2DsfI3u,nA7V5iQqaFcIEQQ0/MKqVdUioieXjPu,None,None,CTfEq6AD/JpKf4ys4awoYB4hpdzgXqm


In [4]:
df=df.transpose()
df.head()

,instructions,ingredients,title,picture_link
p3pKOD6jIHEcjf20CCXohP8uqkG5dGi,Toss ingredients lightly and spoon into a butt...,"[1/2 cup celery, finely chopped, 1 small green...",Grammie Hamblet's Deviled Crab,None
S7aeOIrsrgT0jLP32jKGg4j.o9zi2DO,Watch how to make this recipe.\nSprinkle the s...,"[2 pounds skirt steak, cut into 1/2-inch dice,...",Infineon Raceway Baked Beans,Ja5uaD8Q7m7vvtWwk2.48dr1eCre/qi
o9MItV9txfoPsUQ4v8b0vh1.VdjwfsK,"In a large saucepan, let the beans soak in eno...","[1 1/2 cups dried black beans, picked over and...",Southwestern Black Bean Dip,None
5l1yTSYFifF/M2dfbD6DX28WWQpLWNK,Watch how to make this recipe.\nPreheat the ov...,"[1 1/4 pounds ground chuck, One 15-ounce can t...",Sour Cream Noodle Bake,nm/WxalB.VjEZSa0iX9RuZ8xI51Y7bS
kRBQSWtqYWqtkb34FGeenBSbC32gIdO,Special equipment: sushi mat\nCook the brown r...,"[1 cup rice, brown, medium-grain, cooked, 1/2-...",Sushi Renovation,None


In [5]:
ID = range(len(df))

In [6]:
df['ID']=ID

In [7]:
df.set_index('ID',inplace=True)
df.head()

,instructions,ingredients,title,picture_link
ID,,,,
0,Toss ingredients lightly and spoon into a butt...,"[1/2 cup celery, finely chopped, 1 small green...",Grammie Hamblet's Deviled Crab,None
1,Watch how to make this recipe.\nSprinkle the s...,"[2 pounds skirt steak, cut into 1/2-inch dice,...",Infineon Raceway Baked Beans,Ja5uaD8Q7m7vvtWwk2.48dr1eCre/qi
2,"In a large saucepan, let the beans soak in eno...","[1 1/2 cups dried black beans, picked over and...",Southwestern Black Bean Dip,None
3,Watch how to make this recipe.\nPreheat the ov...,"[1 1/4 pounds ground chuck, One 15-ounce can t...",Sour Cream Noodle Bake,nm/WxalB.VjEZSa0iX9RuZ8xI51Y7bS
4,Special equipment: sushi mat\nCook the brown r...,"[1 cup rice, brown, medium-grain, cooked, 1/2-...",Sushi Renovation,None


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 60039 entries, 0 to 60038
Data columns (total 4 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   instructions  59628 non-null  object
 1   ingredients   59802 non-null  object
 2   title         59750 non-null  object
 3   picture_link  30952 non-null  object
dtypes: object(4)
memory usage: 2.3+ MB


In [9]:
df=df[df['instructions'].isnull()==False]
df=df.reset_index(drop=True)
df=df.loc[:1000]

In [10]:
picture_links = []
folder_path = '/content/drive/MyDrive/Images/'
for picture_link in os.listdir(folder_path):
    if os.path.isfile(os.path.join(folder_path, picture_link)):
        picture_links.append(folder_path+picture_link)
len(picture_links)

1000

In [11]:
picture_links.insert(0, '-')

In [12]:
picture_links.sort()
picture_links.append(picture_links.pop(picture_links.index('-')))

In [13]:
df['picture_link'] = picture_links
df.head()

,instructions,ingredients,title,picture_link
0,Toss ingredients lightly and spoon into a butt...,"[1/2 cup celery, finely chopped, 1 small green...",Grammie Hamblet's Deviled Crab,/content/drive/MyDrive/Images/000001.jpg
1,Watch how to make this recipe.\nSprinkle the s...,"[2 pounds skirt steak, cut into 1/2-inch dice,...",Infineon Raceway Baked Beans,/content/drive/MyDrive/Images/000002.jpg
2,"In a large saucepan, let the beans soak in eno...","[1 1/2 cups dried black beans, picked over and...",Southwestern Black Bean Dip,/content/drive/MyDrive/Images/000003.jpg
3,Watch how to make this recipe.\nPreheat the ov...,"[1 1/4 pounds ground chuck, One 15-ounce can t...",Sour Cream Noodle Bake,/content/drive/MyDrive/Images/000004.jpg
4,Special equipment: sushi mat\nCook the brown r...,"[1 cup rice, brown, medium-grain, cooked, 1/2-...",Sushi Renovation,/content/drive/MyDrive/Images/000005.jpg


In [14]:
def preprocess_txt(line):
    spls = "".join(i for i in line.strip() if i not in exclude).split()
    spls = [morpher.parse(i.lower())[0].normal_form for i in spls]
    spls = [i for i in spls if i not in sw and i != ""]
    return spls

In [15]:
def clean_text(input_text):
  words = []
  for word in input_text.split():
      if not word.isdigit():
          words.append(word)
  clean_text_ = ' '.join(words)
  return clean_text_

In [16]:
assert True

sentences = []

morpher = MorphAnalyzer()
sw = set(get_stop_words("english"))
exclude = set(string.punctuation)
c = 0

with open('/content/drive/MyDrive/Recepts.txt', "r") as fin:
    for line in tqdm_notebook(fin):
        spls=clean_text(line)
        spls = preprocess_txt(spls)
        sentences.append(spls)
        c += 1
        if c > 100000:
            break

<ipython-input-16-decdce07e267>:11: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for line in tqdm_notebook(fin):


0it [00:00, ?it/s]

In [17]:
sentences = [i for i in sentences if len(i) > 2]
sentences[55]

['mix',
 'sift',
 'dry',
 'ingredients',
 'add',
 'milk',
 'melted',
 'shortening',
 'beat',
 'well',
 'bake',
 'slightly',
 'greased',
 'hot',
 'griddle']

In [18]:
modelW2V = Word2Vec(sentences=sentences, vector_size=100, window=5, min_count=1)

In [19]:
modelFT = FastText(sentences=sentences, vector_size=100, min_count=1, window=5, workers=8)

In [20]:
# Создаем Индексы

file_path_from = 'show_recepts.ann'
if not os.path.isfile(file_path_from):


    ft_index_recepts = annoy.AnnoyIndex(100 ,'angular')

    index_map_recepts = {}
    counter = 0

    for i in tqdm_notebook(range(len(df))):
        n_ft = 0
        index_map_recepts[counter] = (df.loc[i, 'title'], df.loc[i, 'instructions'], df.loc[i, 'picture_link'])
        vector_ft = np.zeros(100)
        for word in df.loc[i, "instructions"]:
            if word in modelFT.wv:
                vector_ft += modelFT.wv[word]
                n_ft += 1
        if n_ft > 0:
            vector_ft = vector_ft / n_ft
        ft_index_recepts.add_item(counter, vector_ft)
        counter += 1

    ft_index_recepts.build(10)
    ft_index_recepts.save('show_recepts.ann')

    file_path_from = 'index_recepts.pkl'
    if not os.path.isfile(file_path_from):

      with open("index_recepts.pkl", "wb") as f:
        pickle.dump(index_map_recepts, f)

<ipython-input-20-ed6b871c089d>:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  for i in tqdm_notebook(range(len(df))):


  0%|          | 0/1001 [00:00<?, ?it/s]

In [21]:
ft_index_recepts = annoy.AnnoyIndex(100, 'angular')
ft_index_recepts.load('show_recepts.ann')

index_map_recepts = pd.read_pickle('index_recepts.pkl')

In [22]:
def get_response(users_ingred, index_recepts, map_recepts):
    vector = np.zeros(100)
    norm = 0
    for word in users_ingred:
        if word in modelFT.wv:
            vector += modelFT.wv[word]
            norm += 1
    if norm > 0:
        vector = vector / norm
    answers = index_recepts.get_nns_by_vector(vector, 5, )
    mess=[index_map_recepts[i] for i in answers]
    product_list=[]
    m1=mess[0][0]
    m2=mess[0][1]
    m3=mess[0][2]
    m4=mess[1][0]
    m5=mess[1][1]
    m6=mess[1][2]
    m7=mess[2][0]
    m8=mess[2][1]
    m9=mess[2][2]
    m10=mess[3][0]
    m11=mess[3][1]
    m12=mess[3][2]
    m13=mess[4][0]
    m14=mess[4][1]
    m15=mess[4][2]
    product_list.extend([m1, m2, m3, m4, m5, m6, m7, m8, m9, m10, m11, m12, m13, m14, m15])

    return product_list

In [25]:
bot = telebot.TeleBot("7077676412:AAED9pVdg82wNVXOM7OQ0lXAxQFZTFgyx90")

@bot.message_handler(commands=['start'])
def start(message):
  bot.send_message(message.chat.id, '<b>Hallo! What products do you have</b>', parse_mode='html')

@bot.message_handler(commands=['more_recepts'])
def more_recepts(message):
    new_buttons=types.InlineKeyboardMarkup()
    rec_4=types.InlineKeyboardButton(text='Recept 4', callback_data='Recept 4')
    rec_5=types.InlineKeyboardButton(text='Recept 5', callback_data='Recept 5')
    website=types.InlineKeyboardButton(text='website', callback_data='website')
    new_buttons.add(rec_4, rec_5, website)
    bot.send_message(message.chat.id, 'Press the button to watch more recepts', reply_markup=new_buttons)

@bot.message_handler(content_types=['text'])
def get_user_text(message):
  if message.text:
    markup=types.InlineKeyboardMarkup()
    rec_1=types.InlineKeyboardButton(text='Recept 1', callback_data='Recept 1')
    rec_2=types.InlineKeyboardButton(text='Recept 2', callback_data='Recept 2')
    rec_3=types.InlineKeyboardButton(text='Recept 3', callback_data='Recept 3')
    more_recepts=types.InlineKeyboardButton(text='More recepts', callback_data='More recepts')
    markup.add(rec_1, rec_2, rec_3, more_recepts)
    bot.send_message(message.chat.id, 'Press the button to watch recept', reply_markup=markup)

@bot.callback_query_handler(func=lambda call: True)
def answer(call):
  my_list=get_response(call.message.text, ft_index_recepts, index_map_recepts)
  if call.data == 'Recept 1':
    bot.send_message(call.message.chat.id, my_list[0])
    bot.send_message(call.message.chat.id, my_list[1])
    bot.send_photo(call.message.chat.id, photo=open(my_list[2], 'rb'))
  elif call.data == 'Recept 2':
    bot.send_message(call.message.chat.id, my_list[3])
    bot.send_message(call.message.chat.id, my_list[4])
    bot.send_photo(call.message.chat.id, photo=open(my_list[5], 'rb'))
  elif call.data == 'Recept 3':
    bot.send_message(call.message.chat.id, my_list[6])
    bot.send_message(call.message.chat.id, my_list[7])
    bot.send_photo(call.message.chat.id, photo=open(my_list[8], 'rb'))
  elif call.data == 'More recepts':
    bot.send_message(call.message.chat.id, 'To watch more recepts write /more_recepts')
  elif call.data == 'Recept 4':
    bot.send_message(call.message.chat.id, my_list[9])
    bot.send_message(call.message.chat.id, my_list[10])
    bot.send_photo(call.message.chat.id, photo=open(my_list[11], 'rb'))
  elif call.data == 'Recept 5':
    bot.send_message(call.message.chat.id, my_list[12])
    bot.send_message(call.message.chat.id, my_list[13])
    bot.send_photo(call.message.chat.id, photo=open(my_list[14], 'rb'))
  elif call.data == 'website':
    bot.send_message(call.message.chat.id, 'https://www.foodnetwork.com/')

bot.polling(none_stop=True)